# 株価データの取得と分析

このノートブックでは、PyJQuantsを使った株価データの取得と基本的な分析方法を学びます。

## 目次
1. 環境設定
2. 複数銘柄の比較
3. 財務データの取得
4. 配当情報
5. 市場データ

## 1. 環境設定

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 認証情報の設定（ご自身の情報に書き換えてください）
os.environ['JQUANTS_MAIL_ADDRESS'] = 'your_email@example.com'
os.environ['JQUANTS_PASSWORD'] = 'your_password'

import pyjquants as pjq
print('準備完了')

## 2. 複数銘柄の比較

### 日本の大手自動車メーカーを比較

In [ ]:
# 自動車メーカーの銘柄を取得
auto_makers = {
    '7203': 'トヨタ',
    '7267': 'ホンダ',
    '7201': '日産',
    '7261': 'マツダ',
    '7270': 'SUBARU',
}

# 各銘柄の情報を取得
stocks = {}
for code, name in auto_makers.items():
    stock = pjq.Stock(code)
    stocks[code] = stock
    latest = stock.latest_price
    print(f'{name}({code}): {latest.close:,.0f}円')

### 株価の比較チャート（基準日を100として正規化）

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

for code, stock in stocks.items():
    prices = stock.prices
    # 最初の終値を100として正規化
    normalized = (prices['close'] / prices['close'].iloc[0]) * 100
    ax.plot(prices['date'], normalized, label=f'{auto_makers[code]}', linewidth=2)

ax.set_title('Auto Makers - Normalized Price Comparison (Base=100)', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Normalized Price')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 日次リターンの計算

In [ ]:
# トヨタの日次リターンを計算
toyota = stocks['7203']
prices = toyota.prices.copy()

# 日次リターン（前日比）
prices['daily_return'] = prices['close'].pct_change() * 100

print('直近10日間の日次リターン:')
print(prices[['date', 'close', 'daily_return']].tail(10).to_string(index=False))

print(f'\n平均日次リターン: {prices["daily_return"].mean():.3f}%')
print(f'リターンの標準偏差: {prices["daily_return"].std():.3f}%')

## 3. 財務データの取得

決算情報や財務データも取得できます。

In [ ]:
# トヨタの財務データを取得
financials = toyota.financials

print(f'財務データ: {len(financials)}件')
print(f'カラム: {list(financials.columns)}')

# 直近のデータを表示
if len(financials) > 0:
    financials.head()

## 4. 配当情報

In [ ]:
# 配当情報を取得
dividends = toyota.dividends

print(f'配当データ: {len(dividends)}件')
if len(dividends) > 0:
    print('\n直近の配当情報:')
    dividends.head(10)

## 5. 市場データ

### 取引カレンダー

In [ ]:
from datetime import date

# 市場オブジェクトを作成
market = pjq.Market()

# 特定の日が取引日かどうかを確認
test_dates = [
    date(2024, 12, 25),  # クリスマス
    date(2024, 12, 26),  # 平日
    date(2025, 1, 1),    # 元日
    date(2025, 1, 6),    # 平日
]

print('取引日の確認:')
for d in test_dates:
    is_trading = market.is_trading_day(d)
    status = '取引日' if is_trading else '休場日'
    print(f'{d}: {status}')

### 業種（セクター）一覧

In [ ]:
# 33業種分類を取得
sectors = market.sectors_33

print('33業種分類:')
for i, sector in enumerate(sectors[:10], 1):
    print(f'{i:2}. {sector.code}: {sector.name}')
print(f'... 他 {len(sectors) - 10} 業種')

## まとめ

このノートブックでは以下を学びました：

- 複数銘柄の株価を取得して比較する方法
- 日次リターンの計算方法
- 財務データ・配当データの取得
- 市場カレンダーと業種情報の取得

次のノートブック「03_ペーパートレード.ipynb」では、仮想取引のシミュレーションを行います。